<a href="https://colab.research.google.com/github/Schimmenti/EarthquakesGPS/blob/main/Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import numpy as np
import pandas as pd
import pickle as pkl
import scipy.stats as stats
import matplotlib.pyplot as plt
import datetime
import time
from scipy.ndimage.filters import maximum_filter1d, minimum_filter1d
from google.colab import drive
drive.mount('/content/drive')


def max_filter1d_valid(a, W, add_heading_nan=True):
    hW = (W-1)//2 # Half window size
    if(add_heading_nan):
      return np.concatenate([np.ones(W-1)*np.nan,maximum_filter1d(a,size=W)[hW:-hW]])
    else:
      return maximum_filter1d(a,size=W)[hW:-hW]
def min_filter1d_valid(a, W, add_heading_nan=True):
    hW = (W-1)//2 # Half window size
    if(add_heading_nan):
      return np.concatenate([np.ones(W-1)*np.nan,minimum_filter1d(a,size=W)[hW:-hW]])
    else:
      return minimum_filter1d(a,size=W)[hW:-hW]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
def haversine(lat1, lat2, delta_long):
  return np.sin((lat2-lat1)/2)**2+np.cos(lat1)*np.cos(lat2)*np.sin(delta_long/2)**2

In [63]:
def moving_lin_regress(x, T, t0, t1, add_heading_nan=True):
    m = []
    for t in range(t0,t1-T):
        m.append(stats.linregress(np.arange(0,T),x[t:t+T])[0])
    if(add_heading_nan):
      return np.concatenate([np.ones(T)*np.nan, np.array(m)])
    else:
      return np.array(m)

In [5]:
base_link = "https://raw.githubusercontent.com/Schimmenti/EarthquakesGPS/main/gps_data/"

In [6]:
! wget "https://raw.githubusercontent.com/Schimmenti/EarthquakesGPS/main/gps_data/stat_info.pkl"

--2022-03-10 13:30:09--  https://raw.githubusercontent.com/Schimmenti/EarthquakesGPS/main/gps_data/stat_info.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17030 (17K) [application/octet-stream]
Saving to: ‘stat_info.pkl’

stat_info.pkl       100%[===================>]  16.63K  --.-KB/s    in 0s      

2022-03-10 13:30:09 (64.7 MB/s) - ‘stat_info.pkl’ saved [17030/17030]



In [64]:
with open("stat_info.pkl","rb") as handle:
  station_names, station_pos = pkl.load(handle)

# Processing

In [ ]:
stat_data = {}
for name in station_names:
  try:
    stat_data[name] = pd.read_csv(base_link + name + ".csv")
  except:
    continue

In [ ]:
for name in stat_data.keys():
  #decimal_years  = stat_data[name]["DateD"].values
  #years = np.trunc(decimal_years).astype('int')
  #days_in_year = 365*np.ones(len(years), dtype=int)
  #days_in_year[days_in_year%4==0] += 1
  #days = np.round((decimal_years-years)*days_in_year).astype('int')
  #starting_date = datetime.date(years[0], 1, 1)
  #delta_days = datetime.timedelta(int(days[0]) - 1)
  #starting_date += delta_days
#
  #ending_date = datetime.date(years[-1]-1, 12, 31)
  #delta_days = datetime.timedelta(int(days[-1]))
  #ending_date += delta_days
  #print(ending_date)
  #del stat_data[name]["Date"]

  stat_data[name]['Date'] = pd.to_datetime(stat_data[name]['DateI'], format='%Y%m%d')
  stat_data[name].set_index("Date", drop=True, inplace=True)
  idx = pd.date_range(stat_data[name].index[0], stat_data[name].index[-1])
  stat_data[name] = stat_data[name].reindex(idx, fill_value=np.NaN)

In [ ]:
with open("stat_data.pkl", "wb") as handle:
  pkl.dump(stat_data,handle)

In [ ]:
W=9
coefs = {}
for name in stat_data.keys():
  nord = stat_data[name]["N"].values
  T_w_nord = moving_lin_regress(nord, W, 0, len(nord))
  east = stat_data[name]["E"].values
  T_w_east = moving_lin_regress(east, W, 0, len(east))
  up = stat_data[name]["U"].values
  T_w_up = moving_lin_regress(up, W, 0, len(up))
  coefs[name] = np.array([T_w_nord, T_w_east, T_w_up])

In [ ]:
with open("T_W=%i.pkl"%W, "wb") as handle:
  pkl.dump(coefs,handle)

# Loading

In [65]:
catalog = pd.read_csv("https://raw.githubusercontent.com/Schimmenti/EarthquakesGPS/main/catalog.csv",sep=r"\s+", index_col="Date")

In [9]:
W=9
with open("drive/MyDrive/Colab Notebooks/T_W=%i.pkl"%W,"rb") as handle:
  coefs = pkl.load(handle)
with open("drive/MyDrive/Colab Notebooks/ex_T_W=%i.pkl"%W,"rb") as handle:
  excoefs = pkl.load(handle)

In [10]:
with open("drive/MyDrive/Colab Notebooks/stat_data.pkl","rb") as handle:
  stat_data = pkl.load(handle)

In [79]:
earthquakes = pd.read_csv("https://raw.githubusercontent.com/Schimmenti/EarthquakesGPS/main/sc_4.5_filtered.dat", sep="\s+", header=None, names=["year","month","day", "hh","mn","ss","lat","long","dep","m"])
earthquakes["long"] = -earthquakes["long"]
earthquakes["Date"] =pd.to_datetime(earthquakes[['year','month','day']])
earthquakes.set_index("Date",inplace=True)
earthquakes

,year,month,day,hh,mn,ss,lat,long,dep,m
Date,,,,,,,,,,
1999-05-14,1999,5,14,7,54,2.880000,34.067669,116.367996,6.000000,4.92
1999-06-01,1999,6,1,15,18,2.430000,32.379170,115.225830,12.170000,4.92
1999-08-01,1999,8,1,16,11,22.110001,37.417172,117.033501,3.370000,5.05
1999-08-24,1999,8,24,13,4,6.730000,31.943670,114.554497,24.440001,4.71
1999-10-16,1999,10,16,9,46,43.790001,34.596829,116.273666,2.800000,7.10
...,...,...,...,...,...,...,...,...,...,...
2018-07-05,2018,7,5,15,17,37.139999,35.894669,115.757332,4.650000,4.62
2018-11-19,2018,11,19,20,18,42.689999,32.227169,115.257668,17.620001,4.82
2019-02-25,2019,2,25,18,27,9.990000,30.213671,114.230499,10.340000,4.54


# Excursion

In [ ]:
up_scaling_factor = 4
excoefs = {}
for key in coefs.keys():
  temp = []
  for el in coefs[key]:
    temp.append(max_filter1d_valid(el, W)-min_filter1d_valid(el, W))
  #temp.append(np.sqrt(temp[0]**2+temp[1]**2+(temp[2]/up_scaling_factor)**2))
  temp = np.array(temp)
  excoefs[key] = temp

In [ ]:
with open("ex_T_W=%i.pkl" %W, "wb") as handle:
  pkl.dump(excoefs,handle)

# Catalog

In [50]:
catalog = pd.read_csv("https://raw.githubusercontent.com/Schimmenti/EarthquakesGPS/main/hauksson_relocated.dat",sep=r"\s+",header=None)
catalog.columns = ["sec","m","lat","long","dep"]

In [51]:
landers = catalog[catalog["m"]==7.3]
landers_date = datetime.date(1992,6,28)
landers_time = datetime.timedelta(hours=11, minutes=57, seconds=33)
delta_time = datetime.timedelta(seconds=float(landers['sec'].values))
catalog_beginning = landers_date+landers_time-delta_time
catalog_seconds = catalog['sec'].values.astype('timedelta64[s]')
start_date = np.datetime64(catalog_beginning)
catalog_dates = start_date + catalog_seconds
year_integer =  catalog_dates.astype('datetime64[Y]').astype('int')+1970
month_integer =  catalog_dates.astype('datetime64[M]').astype('int')%12+1
day_integer = (catalog_dates- catalog_dates.astype('datetime64[M]') + 1).astype('timedelta64[D]').astype('int')+1
date_integer =(year_integer*10000+month_integer*100+day_integer)
pandas_datetime = pd.to_datetime(date_integer, format='%Y%m%d')
catalog["Date"] = pandas_datetime
catalog.set_index("Date", drop=True, inplace=True)

In [52]:
catalog.to_csv("catalog.csv", sep="\t")

# Station position

In [80]:
position_array = np.array([station_pos[key] for key in stat_data.keys()])

In [85]:
lat1 = earthquakes['lat'].values.reshape(-1,1)
lat2 = position_array[:,0].reshape(1,-1)
delta_long = earthquakes['long'].values.reshape(-1,1)-position_array[:,1].reshape(1,-1)

In [86]:
distances = haversine(lat1,lat2,delta_long)

In [87]:
closest_station_idx = np.argsort(distances, axis=1)

# Classification

In [127]:
threshold_dist = 5e-3
max_number = 20

In [128]:
for cnt, (date,row) in enumerate(earthquakes.iterrows()):
  for cnt2, key in enumerate(excoefs.keys()):
     dist = distances[cnt,cnt2]
     if(dist <= threshold_dist):

       match = np.argwhere(stat_data[key].index==date).flatten()
       if(len(match)>0):
         match = match[0]
         r_matched = excoefs[key][:,match-W:match]
         print(np.max(np.sum(r_matched[:2,:]**2,axis=0)))
  print("......")

1.2110805555555533
......
......
nan
......
......
......
......
0.6591027777777773
nan
1.8242027777777725
nan
nan
nan
......
nan
......
......
......
......
......
......
......
......
......
nan
0.7507777777777772
......
......
......
......
......
......
0.3634138888888883
0.44384722222222206
0.6228361111111088
0.605136111111109
......
......
......
1.1233694444444418
......
0.2933444444444444
......
0.5689805555555554
nan
......
0.4011805555555557
0.2585694444444438
0.3668472222222225
0.8920277777777774
0.654405555555555
0.19800555555555377
......
0.5335694444444434
......
0.7418055555555542
nan
......
nan
......
......
0.4017111111111141
0.42361111111111116
0.3512555555555541
0.14973888888888884
......
......
......
nan
......
......
nan
......
nan
......
0.4693555555555555
......
0.2943611111111112
0.21362500000000004
0.26269444444444384
0.53894722222222
0.43524722222222345
2.3048111111111123
5.685138888888887
nan
......
1.5561444444444428
......
nan
......
nan
0.5665694444444443

# Discard

In [ ]:
m_day = catalog["m"].groupby("Date").apply(lambda x : (2/3)*np.log10(np.sum(10**(1.5*x))))
true_days = m_day.index
m_day = m_day.values
lat_day = catalog["lat"].groupby("Date").mean().values
long_day = catalog["long"].groupby("Date").mean().values
dep_day = catalog["dep"].groupby("Date").mean().values

In [ ]:
catalog_day = pd.DataFrame(columns = ["m", "lat", "long", "dep" ])

In [ ]:
catalog_day["m"] = m_day
catalog_day["lat"] = lat_day
catalog_day["long"] = long_day
catalog_day["dep"] = dep_day
catalog_day.index = true_days

In [ ]:
catalog_day.to_csv("catalog_day.csv", sep="\t")